# New York Motor Vehicle collisions

## Setup

In [1]:
# import required libraries
import requests
import pandas as pd
import numpy as np
import re
import seaborn as sns

## Data acquision

### define functions to read data

In [2]:
# def read_api_chunk(api, limit=1000, offset=0):
#     source = f"{api}?${limit=}&${offset=}"
#     r = requests.get(source)
#     data = pd.DataFrame.from_dict(r.json()) # read directly from json instead? faster?
#     return data

# use https://docs.python.org/3/reference/expressions.html#yield-expressions


def read_api_chunk(api, limit=1000, offset=0):
    """read a single chunk from the api"""
    return pd.read_json(f"{api}?${limit=}&${offset=}")


def read_api(api, size=1000, chunk_size=1000):
    """read given number of lines from api, applying the chunk_size along the way"""
    chunk_generator = (
        # define chunks; the last chunk might be smaller than chunk_size
        read_api_chunk(api, limit=min(chunk_size, size - x), offset=x)
        for x in range(0, size, chunk_size)
    )
    # in the generator expressions, the chunks are not yet read and stored in memory
    # the outer paranthesis are synctactilly required for generator expressions; they
    # are not included simply in order to permit the multiline definition

    # pd.concat can handle generator expressions. According to the api reference, the objs argument
    # accepts a sequence of DataFrame objects. This indicates that any iterable that yields DataFrame
    # objects will be accepted, which is what chunk_generator provides.
    return pd.concat(chunk_generator)

### inputs

In [3]:
# set input parameters
api = "https://data.cityofnewyork.us/resource/h9gi-nx95.json"
size = 12000
limit = 1000

### read data

In [4]:
# read data
data_raw = read_api_chunk(api, limit=15000)

In [23]:
# initial cleaning of data
data = data_raw.rename(
    columns={
        "vehicle_type_code1": "vehicle_type_code_1",
        "vehicle_type_code2": "vehicle_type_code_2",
    }
)
text_cols = [col for col in data if re.search("(street|contributing_factor)", col)]
data[text_cols] = data[text_cols].astype("string")


In [ ]:
# format data to cover 5 vehicles in one column
data_wide = pd.wide_to_long(
    data,
    stubnames=["vehicle_type_code_", "contributing_factor_vehicle_"],
    i="collision_id",
    j="vehicle_no",
)
data_wide.rename(
    columns={
        "vehicle_type_code_": "vehicle_type_code",
        "contributing_factor_vehicle_": "contributing_factor_vehicle",
    },
    inplace=True,
)

# keep rows for vehicle no. > 1 only if relevant information pertaining to the vehicle is present; the row is redundant otherwise
_cnd1 = (
    data_wide[["vehicle_type_code", "contributing_factor_vehicle"]]
    .notnull()
    .any(axis=1)
)
_cnd2 = data_wide.index.get_level_values(level=1) == 1
_cnd = _cnd1 | _cnd2
data_wide = data_wide.loc[_cnd, :]
# data_wide.dtypes

#### Contributing Factors

In [11]:
# create a mapping for contributing factor to their code
confac = (
    data["contributing_factor_vehicle"]
    .drop_duplicates()
    .dropna()
    .reset_index(drop=True)
    .to_frame(name="contributing_factor")
)


def get_first_chars(input):
    """retrieve first character of each word in a string of words"""
    return "".join(item[0].upper() for item in re.findall("\w+", input))


confac["cf"] = confac["contributing_factor"].apply(get_first_chars)
confac["n"] = confac.groupby(["cf"]).cumcount()
k = confac["n"] > 0
confac.loc[k, "cf"] = confac.loc[k, "cf"] + confac.loc[k, "n"].astype("string")
confac.set_index("contributing_factor", inplace=True)
confac_cols = "cf." + confac["cf"]
mapping_cf = pd.Series(confac["cf"]).to_dict()

In [16]:
# determine dummies grouped by collision_id
data["cf"] = data["contributing_factor_vehicle"].replace(mapping_cf)
dummies_long = pd.get_dummies(data, columns=["cf"], prefix_sep=".")
dummies = dummies_long[confac_cols].groupby(level=0).max()
dummies["n_vehicles"] = dummies.sum(axis=1)
dummies.head()

,cf.ADRR,cf.PS,cf.FTC,cf.U,cf.PTC,cf.DI,cf.POLUI,cf.TI,cf.ULC,cf.US,...,cf.CPHF,cf.LMII,cf.CPHH,cf.UOBND,cf.OED,cf.TCDINW,cf.THD,cf.WI,cf.VV,n_vehicles
collision_id,,,,,,,,,,,,,,,,,,,,,
4136992,False,False,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,2
4277087,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,2
4345591,False,False,True,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,2
4388940,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
4395664,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1


In [ ]:
# join with the original data

In [5]:
cols_contributing_factor = [
    col for col in data if re.match(r"^contributing_factor", col)
]
# pd.Series(data.columns).astype("string").str.startswith("contributing")
# print(cols_contributing_factor)
# data.filter(regex=r"^vehicle", axis=1).isnull().sum(axis=1)
vehicle_type_cols = data.columns.to_series().filter(regex=r"^vehicle").to_list()
data["number_of_vehicles"] = len(vehicle_type_cols) - data[
    vehicle_type_cols
].isnull().sum(axis=1)
contributing_factor_cols = (
    data.columns.to_series().filter(regex=r"^contributing_factor").to_list()
)
data["number_of_contributing_factors"] = len(contributing_factor_cols) - data[
    contributing_factor_cols
].isnull().sum(axis=1)

## Roads

In [6]:
data["on_street_name"].str.split(" ")

0           [WHITESTONE, EXPRESSWAY]
1        [QUEENSBORO, BRIDGE, UPPER]
2             [THROGS, NECK, BRIDGE]
3                               <NA>
4                 [SARATOGA, AVENUE]
                    ...             
14995               [JEROME, AVENUE]
14996       [NORTH, CONDUIT, AVENUE]
14997                           <NA>
14998                    [4, AVENUE]
14999                   [89, STREET]
Name: on_street_name, Length: 15000, dtype: object

## Cyclists

### Injured Cyclists

In [7]:
data["number_of_cyclist_injured"].value_counts(dropna=True)
# data.head(10)

number_of_cyclist_injured
0    14331
1      660
2        9
Name: count, dtype: int64

### Contributing factors

In [8]:
# determine contributing factors
confac_ = pd.concat(
    (data[col] for col in data if re.match("contributing_factor_vehicle", col))
)
confac = confac_.drop_duplicates().dropna().to_frame(name="ContributingFactor")


# get dummies
# filter by number of involved vehicles
# add rolling count
# use mapping
# extract leading characters
def get_first_chars(input):
    """retrieve first character of each word"""
    return "".join(item[0].upper() for item in re.findall("\w+", input))


confac["CF"] = confac["ContributingFactor"].apply(get_first_chars)
confac["CFCount"] = confac.groupby(["CF"]).cumcount()
# confac.loc[confac["MnemoCount"] > 0, ["Mnemo", "MnemoCount"]]
confac.loc[confac["CFCount"] > 0, "CF"] = confac.loc[
    confac["CFCount"] > 0, "CF"
] + confac.loc[confac["CFCount"] > 0, "CFCount"].astype("string")
confac.set_index("ContributingFactor", inplace=True)
confac_mapping = pd.Series(confac["CF"]).to_dict()

cf_cols = [
    re.sub("contributing_factor_vehicle_", "CFV", col)
    for col in contributing_factor_cols
]
data[cf_cols] = data[contributing_factor_cols].replace(confac_mapping)
# pd.get_dummies(data, columns=cf_cols)

In [22]:
data.filter(regex=r"^CF", axis=1)
# pd.get_dummies(data, columns=cf_cols).filter(regex=r"^CF", axis=1)
# use pd.wide_to_long

,CFV1,CFV2,CFV3,CFV4,CFV5
0,ADRR,U,<NA>,<NA>,<NA>
1,PS,<NA>,<NA>,<NA>,<NA>
2,FTC,U,<NA>,<NA>,<NA>
3,U,<NA>,<NA>,<NA>,<NA>
4,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...
14995,DID,PTC,<NA>,<NA>,<NA>
14996,ULC,U,<NA>,<NA>,<NA>
14997,FTYROW,<NA>,<NA>,<NA>,<NA>
14998,DI,DI,<NA>,<NA>,<NA>
